# <center> <img src="../labs/img/ITESOLogo.png" alt="ITESO" width="480" height="130"> </center>
# <center> **Departamento de Electrónica, Sistemas e Informática** </center>
---
## <center> **Procesamiento de Datos Masivos** </center>
---
### <center> **Primavera 2025** </center>
---
### <center> **Lab11 Grupo Foraneos - Ejemplo de Machine Learning with Desicion trees and Support Vector Mchines** </center>

---
**Profesor**: Dr. Pablo Camarillo Ramirez

In [18]:
import findspark
findspark.init()

#### Creacion de la conexión con el cluster de spark


In [19]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("MLSpark-DT_SVM") \
    .master("spark://e9718f8694cf:7077") \
    .config("spark.ui.port","4040") \
    .getOrCreate()
sc = spark.sparkContext
spark.conf.set("spark.sql.shuffle.partitions", "5")

### Creating the schema

In [20]:
from foraneos.spark_utils import SparkUtils
# Create a small dataset as a list of tuples
# Format: (label, feature_x1, feature_x2)

# Define schema for the DataFrame

schema = SparkUtils.generate_schema([("Id", "float"),("SepalLengthCm", "float"),("SepalWidthCm", "float"),("PetalLengthCm", "float"),("PetalWidthCm", "float"),\
                                    ("Species", "string")])


### Loading the Dataset

In [21]:
# Create DataFrame
df = spark \
                .read \
                .schema(schema) \
                .option("header", "true") \
                .option("mode", "dropMalformed")\
                .csv("/home/jovyan/notebooks/data/iris_dataset/iris_ds.csv")
                

### Correcting the label column to numeric values

In [22]:
from pyspark.ml.feature import StringIndexer
label_indexer = StringIndexer(inputCol="Species", outputCol="label") 
data = label_indexer.fit(df).transform(df)

In [23]:
data.show()

+----+-------------+------------+-------------+------------+-----------+-----+
|  Id|SepalLengthCm|SepalWidthCm|PetalLengthCm|PetalWidthCm|    Species|label|
+----+-------------+------------+-------------+------------+-----------+-----+
| 1.0|          5.1|         3.5|          1.4|         0.2|Iris-setosa|  0.0|
| 2.0|          4.9|         3.0|          1.4|         0.2|Iris-setosa|  0.0|
| 3.0|          4.7|         3.2|          1.3|         0.2|Iris-setosa|  0.0|
| 4.0|          4.6|         3.1|          1.5|         0.2|Iris-setosa|  0.0|
| 5.0|          5.0|         3.6|          1.4|         0.2|Iris-setosa|  0.0|
| 6.0|          5.4|         3.9|          1.7|         0.4|Iris-setosa|  0.0|
| 7.0|          4.6|         3.4|          1.4|         0.3|Iris-setosa|  0.0|
| 8.0|          5.0|         3.4|          1.5|         0.2|Iris-setosa|  0.0|
| 9.0|          4.4|         2.9|          1.4|         0.2|Iris-setosa|  0.0|
|10.0|          4.9|         3.1|          1.5|     

### Assemble the features into a single vector column

In [24]:
from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler(inputCols=["SepalLengthCm","SepalWidthCm","PetalLengthCm","PetalWidthCm"], outputCol="features")
data_with_features = assembler.transform(data).select("label", "features")

### Split the data into training and test sets 80% training data and 20% testing data

In [25]:
train_df, test_df = data_with_features.randomSplit([0.8, 0.2], seed=57)

### Show the whole dataset

In [26]:
print("Original Dataset")
data_with_features.show(5)

# Print train dataset
print("train set")
train_df.show(5)

Original Dataset
+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|[5.09999990463256...|
|  0.0|[4.90000009536743...|
|  0.0|[4.69999980926513...|
|  0.0|[4.59999990463256...|
|  0.0|[5.0,3.5999999046...|
+-----+--------------------+
only showing top 5 rows

train set
+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|[4.30000019073486...|
|  0.0|[4.40000009536743...|
|  0.0|[4.40000009536743...|
|  0.0|[4.40000009536743...|
|  0.0|[4.5,2.2999999523...|
+-----+--------------------+
only showing top 5 rows



### Create a Decision Tree model

In [27]:
from pyspark.ml.classification import DecisionTreeClassifier

# Initialize and train the Decision Tree model
dt = DecisionTreeClassifier(labelCol="label", featuresCol="features")

## TRAINNING

In [28]:
dt_model = dt.fit(train_df)

# Display model summary
print("Decision Tree model summary:{0}".format(dt_model.toDebugString))

Decision Tree model summary:DecisionTreeClassificationModel: uid=DecisionTreeClassifier_78e0e2c249b6, depth=5, numNodes=15, numClasses=3, numFeatures=4
  If (feature 2 <= 2.449999988079071)
   Predict: 0.0
  Else (feature 2 > 2.449999988079071)
   If (feature 2 <= 4.8500001430511475)
    If (feature 3 <= 1.6500000357627869)
     Predict: 1.0
    Else (feature 3 > 1.6500000357627869)
     If (feature 0 <= 5.950000047683716)
      Predict: 1.0
     Else (feature 0 > 5.950000047683716)
      Predict: 2.0
   Else (feature 2 > 4.8500001430511475)
    If (feature 3 <= 1.75)
     If (feature 2 <= 4.950000047683716)
      Predict: 1.0
     Else (feature 2 > 4.950000047683716)
      If (feature 3 <= 1.550000011920929)
       Predict: 2.0
      Else (feature 3 > 1.550000011920929)
       Predict: 1.0
    Else (feature 3 > 1.75)
     Predict: 2.0



## PREDICTIONS

In [29]:
# Use the trained model to make predictions on the test data
predictions_DT = dt_model.transform(test_df)

# Show predictions
predictions_DT.select("features", "prediction").show()

+--------------------+----------+
|            features|prediction|
+--------------------+----------+
|[5.0,3.4000000953...|       0.0|
|[5.0,3.5,1.299999...|       0.0|
|[5.09999990463256...|       0.0|
|[5.19999980926513...|       0.0|
|[5.40000009536743...|       0.0|
|[5.69999980926513...|       1.0|
|[5.80000019073486...|       1.0|
|[5.80000019073486...|       1.0|
|[6.09999990463256...|       1.0|
|[6.09999990463256...|       1.0|
|[6.19999980926513...|       1.0|
|[6.30000019073486...|       1.0|
|[6.69999980926513...|       1.0|
|[4.90000009536743...|       1.0|
|[6.0,3.0,4.800000...|       2.0|
|[6.09999990463256...|       2.0|
|[6.19999980926513...|       2.0|
|[6.30000019073486...|       2.0|
|[6.30000019073486...|       2.0|
|[6.40000009536743...|       2.0|
+--------------------+----------+
only showing top 20 rows



# Support Vector machine Example

In [30]:
from pyspark.ml.classification import LinearSVC 
from pyspark.ml.classification import OneVsRest



# Initialize the LinearSVC classifier for binary 
# classification
lsvc = LinearSVC(maxIter=10, regParam=0.01)

# Set up OneVsRest classifier for multi-class 
# classification
ovr = OneVsRest(classifier=lsvc)

## Training

In [31]:
# Train the model
ovr_model = ovr.fit(train_df)

## PREDICTIONS

In [32]:

# Use the trained model to make predictions on the test data
predictions_SVM = ovr_model.transform(test_df)

# Show predictions
#predictions_SVM.select("features", "prediction").show()

# MODEL TESTING

In [33]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator




print("Evaluation of DT: ")
evaluator = MulticlassClassificationEvaluator(labelCol="label",
                            predictionCol="prediction")

accuracy = evaluator.evaluate(predictions_DT, 
                  {evaluator.metricName: "accuracy"})
print(f"Accuracy: {accuracy}")
precision = evaluator.evaluate(predictions_DT,
                  {evaluator.metricName: "weightedPrecision"})
print(f"Precision: {precision}")
recall = evaluator.evaluate(predictions_DT,
                  {evaluator.metricName: "weightedRecall"})
print(f"Recall: {recall}")
f1 = evaluator.evaluate(predictions_DT,
                {evaluator.metricName: "f1"})
print(f"F1 Score: {f1}") 


print("\n Evaluation of SVM:")

evaluator = MulticlassClassificationEvaluator(labelCol="label",
                            predictionCol="prediction")

accuracy = evaluator.evaluate(predictions_SVM, 
                  {evaluator.metricName: "accuracy"})
print(f"Accuracy: {accuracy}")
precision = evaluator.evaluate(predictions_SVM,
                  {evaluator.metricName: "weightedPrecision"})
print(f"Precision: {precision}")
recall = evaluator.evaluate(predictions_SVM,
                  {evaluator.metricName: "weightedRecall"})
print(f"Recall: {recall}")
f1 = evaluator.evaluate(predictions_SVM,
                {evaluator.metricName: "f1"})
print(f"F1 Score: {f1}") 

Evaluation of DT: 
Accuracy: 0.9545454545454546
Precision: 0.9595959595959596
Recall: 0.9545454545454546
F1 Score: 0.9545454545454545

 Evaluation of SVM:


Accuracy: 0.8636363636363636


Precision: 0.8662337662337662
Recall: 0.8636363636363636
F1 Score: 0.8626794258373206


In [34]:
sc.stop()